In [8]:
pip install pyspark

     |████████████████████████████████| 281.3 MB 37 kB/s 
     |████████████████████████████████| 198 kB 51.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=6e0020fcf5d710ec54f01f1acb6fec99de6264ff755b8f5997a6a091f8563426
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [9]:
#importing pyspark library
import pyspark as ps

In [10]:
#setting configaration and creating sql context object 
conf = ps.SparkConf().setAll([('spark.executor.memory', '16g'), ('spark.driver.memory', '16g')])
sc = ps.SparkContext(conf=conf)
from pyspark.sql import SQLContext

sql_context = SQLContext(sc)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [11]:
#connectiong to drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [14]:
#setting data set path
file="/content/gdrive/MyDrive/Colab Notebooks/BigDataProject/tuesday.csv"

In [15]:
#reading csv with headers
tweets = sql_context.read.format('com.databricks.spark.csv').option('header','true').option("delimiter", ",").load(file)

In [16]:
#tweets count
print(tweets.count())

1583509


In [17]:
#display complete tweet first 100
tweets.show(100,truncate=False)


+-------------------------------------------------------------------------------------------------------------------------------------------------------+----+
|Text                                                                                                                                                   |_c1 |
+-------------------------------------------------------------------------------------------------------------------------------------------------------+----+
|RT @kittyshiba: Updated. https://t.co/YUnj6XuBx5                                                                                                       |null|
|RT @SantaDecides: Christmas man Ranboo is on the nice list! https://t.co/L2p9S2XQVV                                                                    |null|
|Check out R-Kive File cardboard sign  https://t.co/aosTpMiAin via @eBay                                                                                |null|
|RT @NBCSports: Players and coaches alike  lis

In [20]:
tweets=tweets.drop("_c1")

In [21]:
#logic to remove RT
tweets.createOrReplaceTempView("tweets")

In [22]:
tweetsWithOutRT = sql_context.sql("SELECT * FROM tweets where text NOT LIKE '%RT%'")

In [ ]:
#tweetsWithOutRT=tweetsWithOutRT.drop("_c1")

In [23]:
tweetsWithOutRT.show()

+--------------------+
|                Text|
+--------------------+
|Check out R-Kive ...|
|Craig and Gizmo r...|
|🕊️🔸You have not...|
|If your group cha...|
|@Kate40870230 @Ro...|
|Fortnite playing ...|
|I wanna play a ga...|
|     36 years old👑.|
|We cannoh replace...|
|all the stochasti...|
|Such class and gr...|
|@AJEnglish Britai...|
|Day 85  run 43  3...|
|hand selected lik...|
|We had productive...|
|Neil Lennon was a...|
|Nanana cover by @...|
|@garethicke THIS ...|
|@ChiThukral @That...|
|“Trump’s Next Cou...|
+--------------------+
only showing top 20 rows



In [24]:
print(tweetsWithOutRT.count())

365186


In [25]:
#preprocesing text
from pyspark.sql.functions import regexp_replace

In [26]:
#removing user names
tweetsWithOutRT = tweetsWithOutRT.withColumn("text",  regexp_replace("text", "@[A-Za-z0-9]+", ""))
 

In [27]:
import pyspark.sql.functions as f

In [28]:
#removing web links and special  characters
tweetsWithOutRT = tweetsWithOutRT.withColumn("text", regexp_replace("text", "(?:\@|http?\://|https?\://|www)\S+", " "))
#tw = tw.withColumn("Tweet", regexp_replace("Tweet", "#[A-Za-z0-9]+", ""))
tweetsWithOutRT = tweetsWithOutRT.withColumn("text", regexp_replace("text", "'", " "))
tweetsWithOutRT = tweetsWithOutRT.withColumn("text", regexp_replace("text", "[^A-Za-z0-9]+", " "))
tweetsWithOutRT = tweetsWithOutRT.withColumn("text", f.lower(f.col("text")) )
tweetsWithOutRT = tweetsWithOutRT.na.drop()

In [29]:
tweetsWithOutRT = tweetsWithOutRT.withColumn("text",regexp_replace("text", "[\\r\\n\\t]", " ") )

In [30]:
tweetsWithOutRT.show()

+--------------------+
|                text|
+--------------------+
|check out r kive ...|
|craig and gizmo r...|
| you have nothing...|
|if your group cha...|
| if i ever watch ...|
|fortnite playing ...|
|i wanna play a ga...|
|       36 years old |
|we cannoh replace...|
|all the stochasti...|
|such class and gr...|
| britain and the ...|
|day 85 run 43 380...|
|hand selected lik...|
|we had productive...|
|neil lennon was a...|
|    nanana cover by |
| this is their pl...|
| quality swag is ...|
| trump s next cou...|
+--------------------+
only showing top 20 rows



In [31]:
#removing null tweets
tweetsWithOutRT = tweetsWithOutRT.where(tweetsWithOutRT.text!='null')

In [32]:
print(tweetsWithOutRT.count())

365186


In [33]:
#tweetspreprocessed.to_csv('tweetspreprocessed.csv', encoding='utf-8')
#saving csv
tweetsWithOutRT.write.csv("/content/gdrive/MyDrive/Colab Notebooks/tuesdaypreprocesedtweets")

In [ ]:
#killing context object
sc.stop()